In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [5]:
##Reading the file

data = pd.read_csv("Churn_modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
##Preprocess the data

##dropping irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## Encode categorical variables

label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
## Onehot encode Geography
'''if we use label encoder for a column like geography then the values wil be assigned like
0 for france, 1 for spain, 2 for germany and while calculations the larger number repressentation
will get more bias. This is why we will use onehot encoding
'''
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
##combining one hot encoded columns with the original data

data=pd.concat([data.drop('Geography',axis = 1), geo_encoded_df],axis=1)

In [12]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [13]:
## We need to save the encoders and sscaler for decoding later
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geography.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [14]:
##diving the dataset into dependent and independent features

X=data.drop('Exited', axis=1)
y=data['Exited']

## Splitting the data in training and testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
X_train.dtype

dtype('float64')

In [16]:
with open('sscaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [17]:
y_train.dtype

dtype('int64')

In [18]:
import torch
from torch import nn
import mlflow
import mlflow.pytorch
import datetime

In [19]:
## Building our model

In [20]:
model=nn.Sequential(
    nn.Linear(in_features=X_train.shape[1],out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64,out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32,out_features=1),
    nn.Sigmoid()
)

In [21]:
optimizer=torch.optim.Adam(params=model.parameters(), lr = 0.005)
loss_fn=torch.nn.BCELoss()

In [22]:
mlflow.set_experiment("exit_prediction")

2025/06/25 06:36:32 INFO mlflow.tracking.fluent: Experiment with name 'exit_prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/ann%20end-to-end%20project/mlruns/743316158203754150', creation_time=1750813592517, experiment_id='743316158203754150', last_update_time=1750813592517, lifecycle_stage='active', name='exit_prediction', tags={}>

In [23]:
epochs=200
## Converting data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [24]:
y_train_tensor.shape

torch.Size([8000])

In [25]:
## The training loop :-

with mlflow.start_run():

    #log hyperparameters
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("lr", 0.005)
    mlflow.log_param("epochs", epochs)

    for epoch in range(epochs):
        model.train()

        #Forward pass
        pred=model(X_train_tensor)
        loss=loss_fn(pred.squeeze(),y_train_tensor)

        #Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Log training loss to MLflow
        mlflow.log_metric("train_loss",loss.item(),step=epoch)

        #print train loss and validation loss
        if(epoch%25 == 0):
            with torch.inference_mode():
                test_pred=model(X_test_tensor)
                test_loss=loss_fn(test_pred.squeeze(),y_test_tensor)
                print(f"the testing loss at epoch {epoch} is {test_loss.item()} the training loss at epoch {epoch} is {loss.item()}")
        

    #Log final trained model to MLflow
    mlflow.pytorch.log_model(model,"final_model")


the testing loss at epoch 0 is 0.6353736519813538 the training loss at epoch 0 is 0.6676588654518127
the testing loss at epoch 25 is 0.428137868642807 the training loss at epoch 25 is 0.4420033097267151
the testing loss at epoch 50 is 0.396981805562973 the training loss at epoch 50 is 0.40857434272766113
the testing loss at epoch 75 is 0.3495784103870392 the training loss at epoch 75 is 0.34941643476486206
the testing loss at epoch 100 is 0.33760446310043335 the training loss at epoch 100 is 0.32233360409736633
the testing loss at epoch 125 is 0.33557751774787903 the training loss at epoch 125 is 0.3092271387577057
the testing loss at epoch 150 is 0.3406088054180145 the training loss at epoch 150 is 0.29538360238075256
the testing loss at epoch 175 is 0.3507213294506073 the training loss at epoch 175 is 0.28301331400871277


2025/06/25 06:36:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/25 06:37:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [26]:
# Define a path to save the model
PATH = "model.pth"

# Save the entire model
torch.save(model, PATH)
print(f"\nModel saved to {PATH}")


Model saved to model.pth


In [27]:
%load_ext mlflow

The mlflow module is not an IPython extension.
